<h1>Webscraping: Extracting and Visualizing Stock Data</h1>
<h3>Description</h3>
<h4>This code extracts stock data of Tesla and graphs it.</h4>

<h2>Table of Contents</h2>
<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ul>
        <li>Defining a Function that Makes a Graph</li>
        <li>yfinance to Extract Stock Data</li>
        <li>Webscraping to Extract Tesla Revenue Data</li>
        <li>Using yfinance to Extract Stock Data</li>
        <li>Using Webscraping to Extract GME Revenue Data</li>
        <li>Plot Tesla Stock Graph</li>
        <li>Plot GameStop Stock Graph</li>
    </ul>
<p>
</div>

<hr>


In [259]:
!pip install yfinance
!pip install bs4
!pip install nbformat
!pip install --upgrade plotly

In [260]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.io as pio
pio.renderers.default = "notebook"
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

## Defining Graphing Function


In [261]:
def make_graph(stock_data, revenue_data, stock):
    """Improved graphing function with data validation and better rendering"""

    # Validate inputs
    if not isinstance(stock_data, pd.DataFrame) or not isinstance(revenue_data, pd.DataFrame):
        raise ValueError("Both stock_data and revenue_data must be pandas DataFrames")

    if 'Date' not in stock_data.columns or 'Close' not in stock_data.columns:
        raise ValueError("stock_data must contain 'Date' and 'Close' columns")

    if 'Date' not in revenue_data.columns or 'Revenue' not in revenue_data.columns:
        raise ValueError("revenue_data must contain 'Date' and 'Revenue' columns")

    # Convert dates to datetime
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])
    revenue_data['Date'] = pd.to_datetime(revenue_data['Date'])

    # Filter data
    stock_data_specific = stock_data[stock_data.Date <= '2021-06-14']
    revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']

    # Create figure
    fig = make_subplots(
        rows=2, cols=1,
        shared_xaxes=True,
        subplot_titles=(
            f"{stock} Historical Share Price",
            f"{stock} Historical Revenue"
        ),
        vertical_spacing=.3
    )

    # Add traces
    fig.add_trace(
        go.Scatter(
            x=stock_data_specific.Date,
            y=stock_data_specific.Close.astype("float"),
            name="Share Price",
            line=dict(color='royalblue', width=2)
        ),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(
            x=revenue_data_specific.Date,
            y=revenue_data_specific.Revenue.astype("float"),
            name="Revenue",
            line=dict(color='green', width=2)
        ),
        row=2, col=1
    )

    # Update layout
    fig.update_layout(
        showlegend=False,
        height=900,
        title=f"{stock} Stock Price and Revenue Analysis",
        plot_bgcolor='white',
        paper_bgcolor='white',
        font=dict(color='black'),
        xaxis_rangeslider_visible=True
    )

    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)

    # Show figure
    fig.show()

## yfinance to Extract Tesla's Stock Data



In [262]:
tesla = yf.Ticker("TSLA")

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `tesla_data`. Set the `period` parameter to ` "max" ` so we get information for the maximum amount of time.


In [263]:
tesla_data = tesla.history(period="max")

In [264]:
tesla_data.reset_index(inplace=True)

tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


# Using Webscraping to Extract Tesla Revenue Data


In [265]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm"
html_data = requests.get(url).text

Parsing the html data using `beautiful_soup` using parser i.e `html5lib` or `html.parser`.


In [266]:
soup = BeautifulSoup(html_data, 'html5lib')

Using `BeautifulSoup` or the `read_html` function extract the table with `Tesla Revenue` and store it into a dataframe named `tesla_revenue`. The dataframe should have columns `Date` and `Revenue`.


In [267]:
tables = soup.find_all('table')
for index, table in enumerate(tables):
    if "Tesla Quarterly Revenue" in str(table):
        table_index = index

tesla_revenue = pd.read_html(str(tables[table_index]), flavor='bs4')[0]
tesla_revenue.columns = ['Date', 'Revenue']

Remove the comma and dollar sign from the `Revenue` column.


In [268]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].astype(str).str.replace('[$,]', '', regex=True)

Remove an null or empty strings in the Revenue column.


In [269]:
tesla_revenue.dropna(inplace=True)

tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

Display the last 5 row of the `tesla_revenue` dataframe using the `tail` function.


In [270]:
print(tesla_revenue.tail())

          Date Revenue
49  2010-06-30      28
50  2010-03-31      21
51  2009-12-31     nan
52  2009-09-30      46
53  2009-06-30      27


## Using yfinance to Extract Stock Data


Using the `Ticker` function enter the ticker symbol of the stock we want to extract data on to create a ticker object. The stock is GameStop and its ticker symbol is `GME`.


In [271]:
gme = yf.Ticker("GME")

Using the ticker object and the function `history` extract stock information and save it in a dataframe named `gme_data`. Set the `period` parameter to ` "max" ` so we get information for the maximum amount of time.


In [272]:
gme_data = gme.history(period="max")


In [273]:
gme_data.reset_index(inplace=True)
gme_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620129,1.693350,1.603296,1.691667,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712708,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


# Webscraping to Extract GME Revenue Data


In [274]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html"
html_data_2 = requests.get(url).text

Parsing the html data using `beautiful_soup` using parser i.e `html5lib` or `html.parser`.


In [275]:
soup = BeautifulSoup(html_data_2, 'html5lib')

Using `BeautifulSoup` or the `read_html` function extract the table with `GameStop Revenue` and store it into a dataframe named `gme_revenue`. The dataframe should have columns `Date` and `Revenue`. Make sure the comma and dollar sign is removed from the `Revenue` column.


In [276]:
gme_revenue = pd.DataFrame(columns=["Date", "Revenue"])

tables = soup.find_all("tbody")
game_stop_table = tables[1]

for row in game_stop_table.find_all("tr"):
    cols = row.find_all("td")
    date = cols[0].text
    revenue = cols[1].text

    gme_revenue = pd.concat([
        gme_revenue,
        pd.DataFrame({"Date": [date], "Revenue": [revenue]})
    ], ignore_index=True)

gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace("$", "").str.replace(",", "")

gme_revenue = gme_revenue[gme_revenue["Revenue"] != ""]
gme_revenue.reset_index(drop=True, inplace=True)

Display the last five rows of the `gme_revenue` dataframe using the `tail` function.

In [277]:
gme_revenue.tail()


,Date,Revenue
57,2006-01-31,1667
58,2005-10-31,534
59,2005-07-31,416
60,2005-04-30,475
61,2005-01-31,709





## Plot Tesla Stock Graph


Using the `make_graph` function to graph the Tesla Stock Data, also provide a title for the graph. Note the graph will only show data upto June 2021.


In [278]:
make_graph(tesla_data, tesla_revenue, 'Tesla')


## Plot GameStop Stock Graph


Using the `make_graph` function to graph the GameStop Stock Data, also provide a title for the graph. The structure to call the `make_graph` function is `make_graph(gme_data, gme_revenue, 'GameStop')`


In [279]:
make_graph(gme_data, gme_revenue, 'GameStop')